# Train model Support Vector Machines

## Prepare data

In [1]:
import pandas as pd
import pickle
from sklearn.svm import SVC
# from sklearn.svm import SVR
from sklearn.metrics import accuracy_score
from sklearn.model_selection import TimeSeriesSplit

INPUT_DATA_FOLDER = "data/source"
ALL_MERGED_DATA = "all_merged_data.csv"

MODEL_FOLDER = "model"
MODEL_FILE_NAME = "support_vector_machines_model.pkl"

FEATURES_TO_INCLUDE = [
    'day_datetime',
    'day_temp',
    'day_humidity',
    'hour_windspeed',
    'hour_conditions',
    'city',
    'event_start_hour',
   #  'event_num_regions',
   # 'event_num_alarms_24h',
    # 'vectors'
]
TARGET_FEATURE = 'is_alarm'

# Load dataset from a CSV file
df = pd.read_csv(f"{INPUT_DATA_FOLDER}/{ALL_MERGED_DATA}", sep=";").head(10000)

# Separate the target variable from the input features
X = df[FEATURES_TO_INCLUDE]
# X = df.drop('event_region_city', axis=1)
y = df[TARGET_FEATURE]

# Convert all columns to float
X = X.apply(pd.to_numeric, errors='coerce')
y = y.apply(pd.to_numeric, errors='coerce')

# Replace NaN values with default
X.fillna(0, inplace=True)
y.fillna(0, inplace=True)

# Split the dataset into training and testing sets
tss = TimeSeriesSplit(2)
train_index, test_index = list(tss.split(X))[-1]
X_train, X_test = X.iloc[train_index, :], X.iloc[test_index,:]
y_train, y_test = y.iloc[train_index], y.iloc[test_index]


## Train model

In [2]:
# Runs for quite a time
# Results in best С = 1
# from sklearn.pipeline import Pipeline
# from sklearn.model_selection import GridSearchCV

# svc = SVC(kernel='linear')

# pipe = Pipeline(steps=[('svc', svc)])

# Cs = list(range(1, 100, 5))
# parameters = dict(svc__C=Cs)

# clf = GridSearchCV(pipe, parameters, verbose=10, cv=3)

# clf.fit(X, y)
# print('Best C:', clf.best_estimator_.get_params()['svc__C'])
# print(); print(clf.best_estimator_.get_params()['svc'])

In [3]:
model = SVC(kernel='linear', C=1)
# model = SVR()
model.fit(X_train, y_train)

# Make predictions on the testing set
y_pred = model.predict(X_test)

# Evaluate the model's accuracy
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)

Accuracy: 0.7620762076207621


## Save model

In [4]:
with open(f"{MODEL_FOLDER}/{MODEL_FILE_NAME}", 'wb') as f:
    pickle.dump(model, f)

## Calculate confusion matrix

In [5]:
from sklearn.metrics import confusion_matrix

# Load the saved model from the file
with open(f"{MODEL_FOLDER}/{MODEL_FILE_NAME}", 'rb') as f:
    model = pickle.load(f)

y_pred = model.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print('Confusion Matrix:')
print(cm)

Confusion Matrix:
[[2540    0]
 [ 793    0]]
